In [1]:
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt

from mqt.qecc.codes.css_code import CSSCode
from mqt.qecc.codes.rotated_surface_code import RotatedSurfaceCode

from mqt.qecc.circuit_synthesis import encoding
from mqt.qecc.circuit_synthesis.noise import (
    CircuitLevelNoise,
    CircuitLevelNoiseIdlingParallel,
    CircuitLevelNoiseIdlingSequential,
    ComposedNoiseModel,
)
from mqt.qecc.circuit_synthesis.simulation import (
    NoisyNDFTStatePrepSimulator,
    VerificationNDFTStatePrepSimulator,
    LutDecoder,
)

import stim  

import sys, importlib
import mqt.qecc as qecc

print("Python:", sys.version)
print("mqt.qecc version:", getattr(qecc, "__version__", "unknown"))

Python: 3.12.12 (main, Oct  9 2025, 11:07:00) [Clang 17.0.0 (clang-1700.3.19.1)]
mqt.qecc version: 1.9.1.dev158+ga3abd71d0
RotatedSurfaceCode available: True


> Notes:
>- CSSCode is the main CSS/stabilizer code abstraction. It exposes check matrices (Hx, Hz), syndrome helpers, and utilities for logicals.
>- Recent versions include RotatedSurfaceCode(distance=...) as a convenience class for the rotated planar family. 
>- Noise and simulation utilities live under mqt.qecc.circuit_synthesis.noise and .simulation. 

# API overview